### Reikiamos bibliotekos ir GPU resursų naudojimo pasirinkimas (jei nėra - naudojama CPU)

In [ ]:
%matplotlib inline

import numpy as np
import torch.nn as nn
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import graph as MyGraph
import time

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
num_workers = 2 if torch.cuda.is_available() else 0
print(f'Device: {device}')

### Esminė treniravimo funkcija ir tikslumo skaičiavimo funkcija

In [ ]:
def train_epoch(optimizer, loss_func, model, loader):
    losses = np.array([], dtype = np.float32)
    
    model.train()
    
    for data in loader:
        # Some manipulation, to make it work on any device.
        images = data[0].to(device)
        labels = torch.nn.functional.one_hot(data[1], model.num_classes).float().to(device) 
        
        # Getting predictions.
        pred = model(images)
        loss = loss_func(pred, labels)
        losses = np.append(losses, loss.cpu().detach().numpy()) 
        
        # Compute the gradient, change values of weights and reset grad to zero for parameters.
        loss.backward()
        optimizer.step()
        optimizer.zero_grad() 
        
    return np.mean(losses)


def evaluate(model, loader):
    correct_count = 0
    total_count = 0
    
    model.eval()
    
    for data in loader:
        images = data[0].to(device)
        labels = data[1].to(device)

        # Enabling no_grad makes it a bit faster.
        with torch.no_grad():
          pred = model(images)
        label_pred = torch.argmax(pred, axis = 1)

        correct_count += torch.sum(labels == label_pred)
        total_count += images.shape[0]
    
    return correct_count / total_count

### Treniravimas ir einamojo duomenų rinkinio tikslumo įvertinimas

`Graph` klasė aprašyta `graph.py` faile. Iškelta, nes ten tik grafikų braižymo klasė, kuri užima daug vietos (vizualiai).

Optimizatorių ir nuostolių funkciją galima keisti būtent čia.

In [ ]:
def train_and_eval(model, loader_train, loader_valid, epoch_count, lr, loss_func, optimizer):
    optimizer = optimizer(model.parameters(), lr)

    graph = MyGraph.Graph()

    for epoch in range(1, epoch_count + 1):
        loss = train_epoch(optimizer, loss_func, model, loader_train) 
        
        train_accuracy = evaluate(model, loader_train)
        valid_accuracy = evaluate(model, loader_valid)    
        
        print(f'[{epoch}] Train Acc: {train_accuracy}.2f  Valid Acc: {valid_accuracy}.2f') 
        
        # Pass accuracies and loss values to graph drawing function.
        graph.draw_graphs(epoch, train_accuracy.cpu().item(), loss, train= True)
        graph.draw_graphs(epoch, valid_accuracy.cpu().item(), None, train= False)

### LeNet modelis, tik vietoj sigmoidinių aktv. funkcijos naudota ELU.
Paimta iš: https://d2l.ai/chapter_convolutional-neural-networks/lenet.html

Dropout taikomas priešpaskutiniam sluoksniui, t. y. atsitktinai pagal nurodytą tikimybė paskutinio sluoksnio neuronai negaus išeičių iš priešpriešpaskutinio sluoksnio.

In [ ]:
class ModelA(nn.Module): 
    def __init__(self, num_classes, dropout_prob):
        super().__init__()
        self.num_classes = num_classes
        self.net = nn.Sequential(
            nn.LazyConv2d(6, kernel_size=5, padding=2), nn.ELU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.LazyConv2d(16, kernel_size=5), nn.ELU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten(),
            nn.LazyLinear(120), nn.ELU(),
            nn.Dropout(p = dropout_prob), nn.LazyLinear(84), nn.ELU(),
            nn.LazyLinear(num_classes))
    
    
    def layer_summary(self, X_shape):
        X = torch.randn(*X_shape)
        for layer in self.net:
            X = layer(X)
            print(layer.__class__.__name__, 'output shape:\t', X.shape)
    
    
    def forward(self, X):
        return self.net(X)
    
    
class ModelB(nn.Module): 
    def __init__(self, num_classes, dropout_prob):
        super().__init__()
        self.num_classes = num_classes
        self.net = nn.Sequential(
            nn.LazyConv2d(6, kernel_size=5, padding=2), nn.ELU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.LazyConv2d(16, kernel_size=7), nn.ELU(),
            nn.Flatten(),
            nn.LazyLinear(100), nn.ELU(),
            nn.LazyLinear(50), nn.ELU(),
            nn.Dropout(p = dropout_prob), nn.LazyLinear(25), nn.ELU(),
            nn.LazyLinear(num_classes))
    
    
    def layer_summary(self, X_shape):
        X = torch.randn(*X_shape)
        for layer in self.net:
            X = layer(X)
            print(layer.__class__.__name__, 'output shape:\t', X.shape)
    
    
    def forward(self, X):
        return self.net(X)
    
    
class ModelC(nn.Module): 
    def __init__(self, num_classes, dropout_prob):
        super().__init__()
        self.num_classes = num_classes
        self.net = nn.Sequential(
            nn.LazyConv2d(6, kernel_size=5, padding=2), nn.ELU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            nn.LazyConv2d(8, kernel_size=3), nn.ELU(),
            nn.MaxPool2d(kernel_size=2, stride=1),
            nn.LazyConv2d(16, kernel_size=3), nn.ELU(),
            nn.Flatten(),
            nn.LazyLinear(80), nn.ELU(),
            nn.Dropout(p = dropout_prob), nn.LazyLinear(24), nn.ELU(),
            nn.LazyLinear(num_classes))
    
    
    def layer_summary(self, X_shape):
        X = torch.randn(*X_shape)
        for layer in self.net:
            X = layer(X)
            print(layer.__class__.__name__, 'output shape:\t', X.shape)
    
    
    def forward(self, X):
        return self.net(X)

### Augmentacijos

In [ ]:
image_w = 32
image_h = 32

train_transforms = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ColorJitter(brightness = 0.4, saturation = 0.1, hue = 0.1),
    transforms.RandomHorizontalFlip(),
    transforms.RandomGrayscale(),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

val_transforms = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])


In [ ]:
directory = "data"
batch_size = 1
dropout_prob = 0.4
num_classes = 10
optimizer = lambda x, l : torch.optim.SGD(x, lr = l)
loss_func = torch.nn.CrossEntropyLoss()
lr = 0.001
epoch_count = 10

# 1 x 3 x 32 x 32 (we are going to use 3-channel pictures with 32 width and 32 height)
image_dimension = [1, 3, image_w, image_h]


train_dataset = torchvision.datasets.ImageFolder(f'{directory}/train', transform= train_transforms)
val_dataset = torchvision.datasets.ImageFolder(f'{directory}/valid', transform= val_transforms)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, num_workers = num_workers, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, num_workers = num_workers, shuffle = False)


model = ModelA(dropout_prob = dropout_prob, num_classes= num_classes).to('cpu')

print('Structure:\n')
model.layer_summary(image_dimension)
print(f'Parameter count: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}')


model.to(device)

start = time.time()
train_and_eval(model, train_loader, val_loader, epoch_count = epoch_count, lr = lr, loss_func=loss_func, optimizer=optimizer)
end = time.time()

print('Time elapsed:', (end - start)/60, " minutes.")

In [ ]:
# torch.save(model.state_dict(), "models/C.model")

### Statsitikų skaičiavimas

In [ ]:
import pandas as pd

classes = {}
for class_name, class_id in val_dataset.class_to_idx.items():
    classes[class_id] = class_name
  
matrix = []
for _ in classes:
    matrix.append([0 for _ in classes])


# Speeds up computations.
with torch.no_grad():
    model.eval()
    for batch, labels in val_loader:
        batch, labels = batch.to(device), labels.to(device)
        predictions = model(batch)
        
        for i in range(len(predictions)):
            predicted_class = torch.argmax(predictions[i])

            if (labels[i] == predicted_class):
                matrix[predicted_class][predicted_class] += 1
            else:
                matrix[labels[i]][predicted_class] += 1



columns = [classes[class_id] for class_id in classes]
dataframe = pd.DataFrame(matrix, columns=columns, index=columns)  

dataframe = pd.concat(
    [pd.concat(
        [dataframe],
        keys=['Predicted Class'], axis=1)],
    keys=['Actual Class']
)

print(dataframe, '\n\n')


## Printing statistics.
for class_id in classes:
    TP = matrix[class_id][class_id]
    FN = sum(matrix[class_id]) - TP
    FP = sum([sublist[class_id] for sublist in matrix]) - TP
    TN = sum(sum(matrix,[])) - TP - FN - FP

    metrics = {}
    metrics['accuracy'] = (TP + TN) / (TP + FP + TN + FN)
    metrics['recall'] = TP / (TP + FN)
    metrics['precision'] = TP / (TP + FP)
    metrics['f1'] = 2 * (metrics['precision'] * metrics['recall']) / (metrics['precision'] + metrics['recall'])

    metrics = {k: round(v, 2) for k, v in metrics.items()}

    # print(TP, FN, FP, TN)
    print(classes[class_id], ':', metrics)


In [ ]:
test_images_30_set = torchvision.datasets.ImageFolder(f'30_test_images/', transform= val_transforms)
test_images_30_loader = torch.utils.data.DataLoader(test_images_30_set, batch_size = batch_size, num_workers = num_workers, shuffle = False)


classes = {}
for class_name, class_id in test_images_30_set.class_to_idx.items():
    classes[class_id] = class_name
  
# Speeds up computations.
with torch.no_grad():
    model.eval()
    
    print('Actual\t\t\tPredicted')
    for batch, labels in test_images_30_loader:
        batch, labels = batch.to(device), labels.to(device)
        predictions = model(batch)
        
        for i in range(len(predictions)):
            predicted_class = torch.argmax(predictions[i]).item()
            
            print(classes[labels[i].item()] + "\t\t\t" + classes[predicted_class])